In [19]:
# Importar los módulos necesarios
import spacy
import os
import glob
import json
import re
import mysql.connector

In [20]:
# Establecer la conexión a la base de datos MySQL
mydb = mysql.connector.connect(
  host="localhost",
  user="juan",
  password="juan",
  database="Practicum"
)

In [21]:
# Cursor para ejecutar consultas SQL
mycursor = mydb.cursor()

In [22]:
# Ruta del directorio de textos
dir_path = 'data'

In [23]:
# Obtener todos los archivos .txt en el directorio
txt_files = glob.glob(os.path.join(dir_path, '*.txt'))

In [24]:
# Cargar el modelo de spaCy
nlp = spacy.load("es_core_news_sm")

In [26]:
# Lista para almacenar nombres de archivos ya insertados
archivos_insertados = []

In [27]:
# Procesar cada archivo de texto
for path in txt_files:
    # Obtener el nombre del archivo
    nombre_archivo = os.path.basename(path)
    
    # Verificar si el nombre del archivo ya ha sido insertado
    if nombre_archivo in archivos_insertados:
        continue  # Si ya está insertado, continuar con el siguiente archivo
    
    archivos_insertados.append(nombre_archivo)  # Agregar a la lista de insertados
    
    # Leer el contenido del archivo
    with open(path, 'r', encoding='utf-8') as file:
        texto = file.read()

    doc = nlp(texto)

    # Insertar el nombre del archivo en la tabla Archivo (solo si no existe)
    try:
        sql_archivo = "INSERT INTO Archivo (nombre_archivo) VALUES (%s)"
        mycursor.execute(sql_archivo, (nombre_archivo,))
        mydb.commit()  # Hacer commit para asegurar la inserción
        id_archivo = mycursor.lastrowid
        
        # Procesar frases nominales y preparar datos
        for frase in doc.noun_chunks:
            frase_sin_puntuacion = re.sub(r'[^\w\s]', '', frase.text).strip()
            palabras = [token.text for token in frase if not token.is_punct and not token.is_space]
            
            # Insertar la frase en la tabla Frases usando el ID de Archivo
            sql_frase = "INSERT INTO Frases (id_archivo, frase) VALUES (%s, %s)"
            frase_values = (id_archivo, frase_sin_puntuacion)
            mycursor.execute(sql_frase, frase_values)
            id_frase = mycursor.lastrowid
            
            # Insertar las palabras en la tabla Palabras y la relación en Frases_palabras
            for palabra in palabras:
                try:
                    # Verificar si la palabra ya existe en Palabras para esta frase
                    sql_select_palabra = "SELECT id_palabra FROM Palabras WHERE palabra = %s AND id_frase = %s"
                    palabra_values = (palabra, id_frase)
                    mycursor.execute(sql_select_palabra, palabra_values)
                    result = mycursor.fetchone()
                    
                    if result:
                        id_palabra = result[0]
                    else:
                        # Insertar la palabra en Palabras si no existe
                        sql_insert_palabra = "INSERT INTO Palabras (palabra, id_frase) VALUES (%s, %s)"
                        mycursor.execute(sql_insert_palabra, (palabra, id_frase))
                        id_palabra = mycursor.lastrowid
                    
                    # No se necesita insertar en Frases_palabras porque ahora Palabras tiene referencia directa a Frases
                    # Si se requiere, sería algo como:
                    # sql_relacion = "INSERT INTO Frases_palabras (id_palabra, id_frase) VALUES (%s, %s)"
                    # relacion_values = (id_palabra, id_frase)
                    # mycursor.execute(sql_relacion, relacion_values)
                
                except mysql.connector.Error as error:
                    print(f"Error: {error}")
            
            # Commit para guardar los cambios de cada frase
            mydb.commit()

    except mysql.connector.Error as error:
        print(f"Error: {error}")

In [28]:
# Cerrar la conexión al finalizar
mydb.close()